<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=98ef6f27635b726e45cf641b3dca80cd2a454f378d7547def0d3a9f0b6b5c780
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 10:06:53
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 1.14 K (0.01%)
Current PnL: -21.23 L (-14.14%)
CY Booked + Current PnL: -9.33 L (-6.21%)
-------------------
Total profit:  1.96 L
Total loss:  -23.18 L
-------------------
Total Booked + Current PnL: 17.65 L (14.34%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.39 L (63.02%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.88,228212.0,17846.0,10931.0,3.25,8.48,4.79,13.68,37.0,1.63,1.64,26.41,XY25,NTT,REFINERIES
18,CIPLA,1662.74,-18.75,62.0,X-LC,3.95,216679.0,12179.0,11116.0,0.25,5.96,5.13,11.39,10.0,1.10,1.56,15.44,X40N,ATH,PHARMA
33,ICICIGI,2260.25,-12.91,67.0,X-MC,2.70,149991.0,13858.0,17264.0,1.29,10.18,11.51,22.86,91.0,0.80,1.08,24.08,X40,ATH,INSURANCE
76,TTKPRESTIG,770.00,94.02,44.0,M-SC,1.74,83578.0,-17199.0,17292.0,-0.65,-17.07,20.69,0.09,245.0,-0.99,0.60,9.80,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.88,56.0,X-MC,1.46,211695.0,19953.0,17804.0,-1.07,10.41,8.41,19.69,99.0,1.12,1.52,17.91,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.88,228212.0,17846.0,10931.0,3.25,8.48,4.79,13.68,37.0,1.63,1.64,26.41,XY25,NTT,REFINERIES
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.41,193696.0,13888.0,98533.0,2.30,7.72,50.87,62.52,88.0,0.14,1.39,44.44,XR,NTT,BANKS
78,UNIONBANK,163.00,-11.63,56.0,M-LC,8.71,158896.0,18056.0,26440.0,2.24,12.82,16.64,31.59,66.0,0.68,1.14,42.70,XY24,NTT,BANKS
43,JIOFIN,387.00,-0.34,57.0,X-LC,11.85,270857.0,3492.0,61972.0,1.83,1.31,22.88,24.48,48.0,0.06,1.95,57.00,XY24,BTT,FINANCE
45,KPIGREEN,731.05,2.88,62.0,H-SC,5.91,122711.0,-2586.0,61515.0,1.78,-2.06,50.13,47.03,141.0,-0.04,0.88,53.65,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-40.66,61.0,M-SC,4.19,100365.0,8866.0,69884.0,-3.34,9.69,69.63,86.07,223.0,0.13,0.72,45.75,XR,NTT,DURABLES
31,HINDZINC,730.22,29.80,52.0,M-LC,10.09,211084.0,6008.0,106555.0,-3.00,2.93,50.48,54.89,52.0,0.06,1.52,27.73,X5K,ATH,METALS
25,GREENPANEL,537.00,229.46,54.0,M-SC,4.16,147010.0,-36068.0,115050.0,-2.89,-19.70,78.26,43.14,230.0,-0.31,1.06,38.28,XY24,NTT,MISC
70,TANLA,1963.11,-25.26,40.0,H-SC,18.83,200889.0,-69872.0,399829.0,-2.68,-25.81,199.03,121.86,133.0,-0.17,1.44,57.15,AR,ATH,IT
57,REPCOHOME,880.00,-58.27,55.0,H-SC,2.49,238950.0,-47555.0,304900.0,-1.93,-16.60,127.60,89.82,134.0,-0.16,1.72,26.42,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-56.32,35.0,H-SC,7.72,122197.0,-1139.0,132767.0,-1.64,-0.92,108.65,106.72,119.0,-0.01,0.88,22.24,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,110.04,62.0,H-SC,12.72,132678.0,-9801.0,29322.0,0.86,-6.88,22.10,13.70,163.0,-0.33,0.95,56.60,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.24,57.0,H-SC,1.88,225828.0,-43839.0,80169.0,-0.92,-16.26,35.50,13.47,138.0,-0.55,1.62,14.83,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,94.02,44.0,M-SC,1.74,83578.0,-17199.0,17292.0,-0.65,-17.07,20.69,0.09,245.0,-0.99,0.60,9.80,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.34,38.0,H-SC,2.50,137816.0,-38087.0,80112.0,-0.17,-21.65,58.13,23.89,149.0,-0.48,0.99,19.02,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.05,46.0,H-MC,7.51,102666.0,-29169.0,71394.0,0.63,-22.13,69.54,32.03,98.0,-0.41,0.74,15.00,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.04,207488.0,4318.0,37804.0,0.61,2.13,18.22,20.74,55.0,0.11,1.49,7.79,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.80,52.0,M-LC,10.09,211084.0,6008.0,106555.0,-3.00,2.93,50.48,54.89,52.0,0.06,1.52,27.73,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,56.0,M-LC,8.71,158896.0,18056.0,26440.0,2.24,12.82,16.64,31.59,66.0,0.68,1.14,42.70,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.41,193696.0,13888.0,98533.0,2.30,7.72,50.87,62.52,88.0,0.14,1.39,44.44,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.66,61.0,M-SC,4.19,100365.0,8866.0,69884.0,-3.34,9.69,69.63,86.07,223.0,0.13,0.72,45.75,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-25.47,59.0,H-MC,12.41,193696.0,13888.0,98533.0,2.30,7.72,50.87,62.52,88.0,0.14,1.39,44.44,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.66,61.0,M-SC,4.19,100365.0,8866.0,69884.0,-3.34,9.69,69.63,86.07,223.0,0.13,0.72,45.75,XR,NTT,DURABLES
31,HINDZINC,730.22,29.80,52.0,M-LC,10.09,211084.0,6008.0,106555.0,-3.00,2.93,50.48,54.89,52.0,0.06,1.52,27.73,X5K,ATH,METALS
36,INDIAMART,4810.62,-56.32,35.0,H-SC,7.72,122197.0,-1139.0,132767.0,-1.64,-0.92,108.65,106.72,119.0,-0.01,0.88,22.24,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.01,149228.0,-1717.0,111175.0,-0.09,-1.14,74.50,72.51,53.0,-0.02,1.07,4.82,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,145109.0,-131414.0,241490.0,0.79,-47.52,166.42,39.81,54.0,-0.54,1.04,2.28,XY24,NTT,AUTO
51,QUESS,986.00,-48.76,31.0,X-SC,38.81,50983.0,-14023.0,164950.0,-0.56,-21.57,323.54,232.18,198.0,-0.09,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.31,40.0,X-MC,2.41,229757.0,-17974.0,93029.0,-1.25,-7.26,40.49,30.30,92.0,-0.19,1.65,5.07,X40,ATH,ELECTRICAL
55,RELAXO,1176.00,-39.88,42.0,X-SC,4.56,81949.0,-63211.0,139141.0,-0.54,-43.55,169.79,52.31,136.0,-0.45,0.59,9.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.01,43.0,X-SC,4.30,95616.0,-33054.0,78348.0,0.15,-25.69,81.94,35.20,219.0,-0.42,0.69,9.75,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.61,47.0,H-LC,0.38,156675.0,-29420.0,76896.0,0.39,-15.81,49.08,25.51,15.0,-0.38,1.13,15.46,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.88,56.0,X-MC,1.46,211695.0,19953.0,17804.0,-1.07,10.41,8.41,19.69,99.0,1.12,1.52,17.91,XY25,NTT,AC
28,HAVELLS,2069.16,-7.31,40.0,X-MC,2.41,229757.0,-17974.0,93029.0,-1.25,-7.26,40.49,30.30,92.0,-0.19,1.65,5.07,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,61.0,X-LC,2.59,202397.0,2259.0,18641.0,0.42,1.13,9.21,10.44,4.0,0.12,1.45,6.76,X40,NTT,FMCG
33,ICICIGI,2260.25,-12.91,67.0,X-MC,2.70,149991.0,13858.0,17264.0,1.29,10.18,11.51,22.86,91.0,0.80,1.08,24.08,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,31.0,X-SC,38.81,50983.0,-14023.0,164950.0,-0.56,-21.57,323.54,232.18,198.0,-0.09,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,145109.0,-131414.0,241490.0,0.79,-47.52,166.42,39.81,54.0,-0.54,1.04,2.28,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,43.0,X-MC,3.23,182810.0,-55141.0,207782.0,-0.25,-23.17,113.66,64.15,174.0,-0.27,1.31,2.55,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.69,43.0,X-MC,5.45,179564.0,-20942.0,118099.0,0.06,-10.44,65.77,48.45,90.0,-0.18,1.29,3.55,X40N,ATH,FINANCE
73,TCS,4389.97,-29.33,49.0,X-LC,12.75,281615.0,-64341.0,131035.0,1.14,-18.60,46.53,19.28,1.0,-0.49,2.02,4.11,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,31.0,X-SC,38.81,50983.0,-14023.0,164950.0,-0.56,-21.57,323.54,232.18,198.0,-0.09,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-39.88,42.0,X-SC,4.56,81949.0,-63211.0,139141.0,-0.54,-43.55,169.79,52.31,136.0,-0.45,0.59,9.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.01,43.0,X-SC,4.30,95616.0,-33054.0,78348.0,0.15,-25.69,81.94,35.20,219.0,-0.42,0.69,9.75,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.27,47.0,X-SC,0.84,106686.0,4682.0,39239.0,0.38,4.59,36.78,43.06,122.0,0.12,0.77,22.49,X40N,ATH,MISC
32,HONAUT,58357.33,-22.63,48.0,X-SC,2.55,108990.0,-18948.0,66081.0,0.06,-14.81,60.63,36.84,143.0,-0.29,0.78,11.58,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.33,49.0,X-LC,12.75,281615.0,-64341.0,131035.0,1.14,-18.60,46.53,19.28,1.0,-0.49,2.02,4.11,X40,ATH,IT
39,INFY,2275.00,-19.96,46.0,X-LC,7.62,311726.0,-1162.0,172852.0,1.55,-0.37,55.45,54.87,3.0,-0.01,2.24,6.25,X40,BTT,IT
41,ITC,452.00,-37.50,61.0,X-LC,2.59,202397.0,2259.0,18641.0,0.42,1.13,9.21,10.44,4.0,0.12,1.45,6.76,X40,NTT,FMCG
82,VBL,671.64,-18.84,51.0,X-LC,10.51,292172.0,-23670.0,136327.0,-0.78,-7.49,46.66,35.67,5.0,-0.17,2.10,5.32,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,53.0,H-LC,6.89,251472.0,-10147.0,129357.0,0.71,-3.88,51.44,45.57,7.0,-0.08,1.81,7.00,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6855.56,61.0,L-SC,6.41,76344.0,-17466.0,94086.0,0.95,-18.62,123.24,81.67,269.0,-0.19,0.55,47.84,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,29.66,82560.0,-30989.0,71043.0,1.49,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,43.0,H-SC,9.42,90060.0,-7920.0,29522.0,0.94,-8.08,32.78,22.05,152.0,-0.27,0.65,30.91,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-40.66,61.0,M-SC,4.19,100365.0,8866.0,69884.0,-3.34,9.69,69.63,86.07,223.0,0.13,0.72,45.75,XR,NTT,DURABLES
45,KPIGREEN,731.05,2.88,62.0,H-SC,5.91,122711.0,-2586.0,61515.0,1.78,-2.06,50.13,47.03,141.0,-0.04,0.88,53.65,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-187.50,71.0,M-SC,19.67,151920.0,2700.0,141483.0,1.22,1.81,93.13,96.62,208.0,0.02,1.09,64.41,XY25,NTT,FINANCE
4,ANGELONE,3033.0,15.82,70.0,X-SC,8.29,205213.0,14207.0,43485.0,0.24,7.44,21.19,30.21,157.0,0.33,1.47,29.87,X40N,NTT,FINANCE
67,STARHEALTH,761.0,21.26,67.0,H-SC,9.79,270205.0,-10918.0,143776.0,-1.44,-3.88,53.21,47.26,144.0,-0.08,1.94,44.77,XY24,NTT,INSURANCE
66,SONACOMS,1006.0,-34.97,64.0,M-SC,11.43,80981.0,-20178.0,95072.0,0.12,-19.95,117.40,74.03,202.0,-0.21,0.58,14.19,AR,BTT,AUTO
77,UJJIVANSFB,60.0,110.04,62.0,H-SC,12.72,132678.0,-9801.0,29322.0,0.86,-6.88,22.10,13.70,163.0,-0.33,0.95,56.60,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,44.33
2,50,75.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.19
LC    34.04
MC    21.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.53
X40      14.87
XY25     12.45
XR       10.94
X40N     10.25
AR        8.02
OX40N     7.62
X200      1.83
X5K       1.52
SR        1.12
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.32
X-LC    23.02
X-MC    16.08
M-SC    12.72
M-LC     5.22
H-LC     4.77
X-SC     4.67
H-MC     3.83
M-MC     1.58
L-SC     1.48
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-3.40,37.48
IT,12.74,-18.51,82.47
FINANCE,12.19,-11.47,62.00
MISC,6.85,-20.88,87.86
BANKS,6.41,-11.89,70.75
PAINTS,5.71,-16.18,33.50
ELECTRICAL,5.37,-10.24,48.92
INSURANCE,3.99,0.30,36.97
AUTO,2.82,-46.39,116.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3326092.0,22
XR,1388418.0,14
AR,1213595.0,9
X40,766708.0,10
X40N,601533.0,8
OX40N,587241.0,10
XY25,450266.0,8
SR,266253.0,2
X5K,106555.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3478170.0,25
M-SC,1271901.0,15
X-MC,1089483.0,11
X-LC,1034821.0,13
X-SC,531244.0,6
H-MC,346624.0,3
M-LC,281974.0,4
H-LC,277048.0,3
L-SC,255640.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197310.0      6
           AR         844749.0      5
           XR         834239.0      7
M-SC       XY24       696816.0      6
X-MC       XY24       588141.0      4
X-LC       X40        471740.0      5
           XY24       303462.0      2
X-SC       X40N       287946.0      4
H-SC       OX40N      274104.0      4
           SR         266253.0      2
X-MC       X40        216620.0      4
H-LC       AR         206253.0      2
X-LC       X40N       195488.0      3
H-MC       XY24       176697.0      1
M-MC       XY24       172276.0      1
X-MC       XY25       166623.0      2
L-SC       XR         165129.0      2
X-SC       XY24       164950.0      1
M-SC       AR         162593.0      2
           OX40N      151232.0      4
           XY25       141483.0      1
           XR         119777.0      2
X-MC       X40N       118099.0      1
M-LC       XR         111175.0      1
           X5K        106555.0      1
H-MC       XR          98533.0      1
L-SC       OX40N       90511.0      1
X-SC       X40         78348.0      1
H-MC       OX40N       71394.0      1
H-LC       X200        70795.0      1
X-LC       XY25        64131.0      3
H-SC       MH          61515.0      1
L-MC       XR          59565.0      1
L-LC       XY25        40225.0      1
M-LC       XY25        37804.0      1
           XY24        26440.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
